In [179]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine,inspect, func

import pandas as pd
import numpy as np

In [180]:
# Create engine using the RestonDev.sqlite database file

engine = create_engine("sqlite:///static/resources/data/RestonDev.sqlite")

In [181]:
# Create a connection to the engine called `conn`
conn = engine.connect()
# Declare a Base using `automap_base()`
Base = automap_base()
# Use the Base class to reflect the database tables
Base.prepare(engine, reflect=True)
# Print all of the classes mapped to the Base
Base.classes.keys()

['ExApp', 'ExURApp', 'Existing', 'PlanMax']

In [182]:
# Assign the Measurement class, Station class to a variables called Measurement and Station respectively
Existing = Base.classes.Existing
PlanMax = Base.classes.PlanMax
ExApp = Base.classes.ExApp
ExURApp = Base.classes.ExURApp

In [183]:
# Create a session
session = Session(engine)

In [184]:
#Testing Queries

#TSA = "Herndon TSA"
#DistSubDist = "Herndon Station"
#LUCategory = "Transit Station Mixed Use"

TSA = "Herndon TSA"
DistSubDist = ""
LUCategory = ""

EX_FilterString = ""
PlanMax_FilterString = ""
EXApp_FilterString = ""
EXURApp_FilterString = ""


if TSA != "":
    EX_FilterString = "Existing.TSA == '" + TSA + "'"
    PlanMax_FilterString = "PlanMax.TSA == '" + TSA + "'"
    EXApp_FilterString = "EXApp.TSA == '" + TSA + "'"
    EXURApp_FilterString = "EXURApp.TSA == '" + TSA + "'"
    
if DistSubDist != "":
    if TSA != "":
        EX_FilterString = EX_FilterString +" AND Existing.Dis_SubDis == '" + DistSubDist + "'"
        PlanMax_FilterString = PlanMax_FilterString +" AND PlanMax.Dis_SubDis == '" + DistSubDist + "'"
        EXApp_FilterString = EXApp_FilterString +" AND EXApp.Dis_SubDis == '" + DistSubDist + "'"
        EXURApp_FilterString = EXURApp_FilterString +" AND EXURApp.Dis_SubDis == '" + DistSubDist + "'"
    else:
        EX_FilterString ="Existing.Dis_SubDis == '" + DistSubDist + "'"
        PlanMax_FilterString = "PlanMax.Dis_SubDis == '" + DistSubDist + "'"
        EXApp_FilterString = "EXApp.Dis_SubDis == '" + DistSubDist + "'"
        EXURApp_FilterString = "EXURApp.Dis_SubDis == '" + DistSubDist + "'"
    
if LUCategory != "":
    if TSA != "" or DistSubDist != "":
        EX_FilterString = EX_FilterString +" AND Existing.LUCategory == '" + LUCategory + "'"
        PlanMax_FilterString = PlanMax_FilterString +" AND PlanMax.LUCategory == '" + LUCategory + "'"
        EXApp_FilterString = EXApp_FilterString +" AND EXApp.LUCategory == '" + LUCategory + "'"
        EXURApp_FilterString = EXURApp_FilterString +" AND EXURApp.LUCategory == '" + LUCategory + "'"
    elif TSA == "" and DistSubDist == "":
        EX_FilterString = "Existing.LUCategory == '" + LUCategory + "'"
        PlanMax_FilterString = "PlanMax.LUCategory == '" + LUCategory + "'"
        EXApp_FilterString = "EXApp.LUCategory == '" + LUCategory + "'"
        EXURApp_FilterString = "EXURApp.LUCategory == '" + LUCategory + "'"
        
    
print(EX_FilterString) 
print(PlanMax_FilterString) 
print(EXApp_FilterString) 
print(EXURApp_FilterString) 

ExistingData = session.query(Existing.Scenario,func.sum(Existing.Office).label('Office'),\
                            func.sum(Existing.Retail).label('Retail'),\
                            func.sum(Existing.Hotel).label('Hotel'),\
                            func.sum(Existing.Institutional).label('Institutional'),\
                            func.sum(Existing.Industrial).label('Industrial'),\
                            func.sum(Existing.Nonresidential_GFA).label('Nonresidential_GFA'),\
                            func.sum(Existing.Residential_GFA).label('Residential_GFA'),\
                            func.sum(Existing.Residential_Units).label('Residential_Units')).filter(EX_FilterString).\
                            group_by(Existing.Scenario).all()
                            
PlanMaxData = session.query(PlanMax.Scenario,func.sum(PlanMax.Office).label('Office'),\
                            func.sum(PlanMax.Retail).label('Retail'),\
                            func.sum(PlanMax.Hotel).label('Hotel'),\
                            func.sum(PlanMax.Institutional).label('Institutional'),\
                            func.sum(PlanMax.Industrial).label('Industrial'),\
                            func.sum(PlanMax.Nonresidential_GFA).label('Nonresidential_GFA'),\
                            func.sum(PlanMax.Residential_GFA).label('Residential_GFA'),\
                            func.sum(PlanMax.Residential_Units).label('Residential_Units')).filter(PlanMax_FilterString).\
                            group_by(PlanMax.Scenario).all()

ExAppData = session.query(ExApp.Scenario,func.sum(ExApp.Office).label('Office'),\
                            func.sum(ExApp.Retail).label('Retail'),\
                            func.sum(ExApp.Hotel).label('Hotel'),\
                            func.sum(ExApp.Institutional).label('Institutional'),\
                            func.sum(ExApp.Industrial).label('Industrial'),\
                            func.sum(ExApp.Nonresidential_GFA).label('Nonresidential_GFA'),\
                            func.sum(ExApp.Residential_GFA).label('Residential_GFA'),\
                            func.sum(ExApp.Residential_Units).label('Residential_Units')).filter(EXApp_FilterString).\
                            group_by(ExApp.Scenario).all()
        
ExURAppData = session.query(ExURApp.Scenario,func.sum(ExURApp.Office).label('Office'),\
                            func.sum(ExURApp.Retail).label('Retail'),\
                            func.sum(ExURApp.Hotel).label('Hotel'),\
                            func.sum(ExURApp.Institutional).label('Institutional'),\
                            func.sum(ExURApp.Industrial).label('Industrial'),\
                            func.sum(ExURApp.Nonresidential_GFA).label('Nonresidential_GFA'),\
                            func.sum(ExURApp.Residential_GFA).label('Residential_GFA'),\
                            func.sum(ExURApp.Residential_Units).label('Residential_Units')).filter(EXURApp_FilterString).\
                            group_by(ExURApp.Scenario).all()
                                                      
ExistingDataDF = pd.DataFrame(ExistingData)
PlanMaxDataDF = pd.DataFrame(PlanMaxData)
ExAppDataDF = pd.DataFrame(ExAppData)
ExURAppDataDF = pd.DataFrame(ExURAppData)

Testframes = [ExistingDataDF, PlanMaxDataDF, ExAppDataDF,ExURAppDataDF]

result = pd.concat(Testframes)

#SampleExistingDataDF.head()
result.head()
#sampleIDDict = sampleMetaDataDF.to_dict('records')

    

Existing.TSA == 'Herndon TSA'
PlanMax.TSA == 'Herndon TSA'
EXApp.TSA == 'Herndon TSA'
EXURApp.TSA == 'Herndon TSA'


C:\Users\Ind\Anaconda3\envs\PythonData_3\lib\site-packages\sqlalchemy\sql\elements.py:4230: SAWarning: Textual SQL expression "Existing.TSA == 'Herndon ..." should be explicitly declared as text("Existing.TSA == 'Herndon ...") (this warning may be suppressed after 10 occurrences)
  {"expr": util.ellipses_string(element)})
C:\Users\Ind\Anaconda3\envs\PythonData_3\lib\site-packages\sqlalchemy\sql\elements.py:4230: SAWarning: Textual SQL expression "PlanMax.TSA == 'Herndon T..." should be explicitly declared as text("PlanMax.TSA == 'Herndon T...") (this warning may be suppressed after 10 occurrences)
  {"expr": util.ellipses_string(element)})
C:\Users\Ind\Anaconda3\envs\PythonData_3\lib\site-packages\sqlalchemy\sql\elements.py:4230: SAWarning: Textual SQL expression "EXApp.TSA == 'Herndon TSA..." should be explicitly declared as text("EXApp.TSA == 'Herndon TSA...") (this warning may be suppressed after 10 occurrences)
  {"expr": util.ellipses_string(element)})


,Scenario,Office,Retail,Hotel,Institutional,Industrial,Nonresidential_GFA,Residential_GFA,Residential_Units
0,Existing Development,2546267,128581,113720,7014,0,2795582,3369741,2670
0,Plan Buildout,5182094,226121,584985,23239,0,6016439,8523200,8175
0,Existing + Approved,3106267,148581,113720,7014,0,2795582,4912246,3732
0,Existing + Approved + UnderReview,3503352,185081,113720,7014,0,2795582,4912246,3732
